# Stanford Dogs - A Classfication problem

Classification is a fundamental task in machine learning, and the Stanford Dogs Dataset provides a valuable resource for training and evaluating classification models. The dataset consists of images of various dog breeds, each labeled with the corresponding breed.

By leveraging this dataset, we can develop a classification model that can accurately identify the breed of a given dog image. This can have practical applications in areas such as pet identification, animal welfare, and breed-specific research.

To build a classification model using the Stanford Dogs Dataset, we can employ various machine learning techniques, such as convolutional neural networks (CNNs). CNNs are particularly effective for image classification tasks, as they can automatically learn relevant features from the input images.

By training a CNN on the Stanford Dogs Dataset, we can teach the model to recognize distinctive patterns and characteristics of different dog breeds. Once trained, the model can be used to classify new dog images, providing predictions about the breed with a certain level of confidence.

Evaluation of the classification model can be done using metrics such as accuracy, precision, recall, and F1 score. These metrics help assess the model's performance and determine its effectiveness in correctly classifying dog breeds.

Overall, the Stanford Dogs Dataset offers a valuable opportunity to explore and develop classification models for dog breed identification. By leveraging this dataset and employing appropriate machine learning techniques, we can contribute to the field of computer vision and enhance our understanding of dog breeds.

## 00 - Preprocessing

Since we are using the **Stanford Dogs** dataset, there aren't really must preprocessing other than loading the data provides to us. - (What he trying to say???? >.< )

In [30]:
import tensorflow as tf
import xml.etree.ElementTree as ET
import os

# * Parses the annotation XML file to extract the bounding box coordinates
def parse_xml(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    bndbox = root.find('.//bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)
    return xmin, ymin, xmax, ymax

# * Loads the image and crops it based on the bounding box coordinates
def load_image_and_crop(path, annotation_path):
    # Decode paths
    path = path.numpy().decode('utf-8')
    annotation_path = annotation_path.numpy().decode('utf-8')
    
    xmin, ymin, xmax, ymax = parse_xml(annotation_path)
    
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = image[ymin:ymax, xmin:xmax]
    return image

# * Preprocesses the image by resizing and normalizing it
def preprocess_image(image):
    image = tf.image.resize(image, [224, 224])
    image = (image / 255.0) - 0.5  # Normalize
    return image

# * Combines the above functions to load and preprocess the image
def load_and_preprocess_image(path, annotation_path):
    image = tf.py_function(func=load_image_and_crop, inp=[path, annotation_path], Tout=tf.uint8)
    image.set_shape([None, None, 3])
    image = preprocess_image(image)
    return image

# * Prepares the dataset by loading and preprocessing the images
def prepare_dataset(image_dir, annotation_dir):
    image_paths = []
    annotation_paths = []
    
    # Adjust the code to account for subdirectories
    for breed_dir in os.listdir(image_dir):
        breed_image_dir = os.path.join(image_dir, breed_dir)
        breed_annotation_dir = os.path.join(annotation_dir, breed_dir)
        for image_file in os.listdir(breed_image_dir):
            image_path = os.path.join(breed_image_dir, image_file)
            annotation_path = os.path.join(breed_annotation_dir, image_file.split('.')[0] + '.xml')
            image_paths.append(image_path)
            annotation_paths.append(annotation_path)
    
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, annotation_paths))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

# - Test the functions
image_dir = 'data/images'
annotation_dir = 'data/annotation-xml'

# * Initialize the dataset
dataset = prepare_dataset(image_dir, annotation_dir)

# * Print the shape of the first 5 images
for image in dataset.take(5):
    print(image.shape)  # This should now print [224, 224, 3]


(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
